In [1]:
import sys
sys.path.append('c:/Users/rmmoh/code/rmm/advent-rmm-2023')

In [2]:
import numpy as np
import pandas as pd
import pydot
from operator import itemgetter
from collections import Counter
import datetime

In [18]:
fh = open("input25-1.dat")
#fh = open("test.dat")
#fh = open("test2.dat")

lines = fh.readlines()
lines = [x.strip("\n") for x in lines]

class Graph:
    def __init__(self, skip_edges=None):
        self.edges = set()
        self.nodes = {}
        if skip_edges is None:
            self.skip_edges = []
        else:
            self.skip_edges = skip_edges

    def process_str(self, initstr):
        spl = initstr.split(": ")
        source = spl[0]
        ends = spl[1].split(" ")
        for e in ends:
            if ((source, e) not in self.skip_edges) and ((e, source) not in self.skip_edges):
                self.add_edge( (source, e) )

    def add_edge(self, edge_tup):
        self.edges.add( edge_tup )

    def finalize(self):
        for (a,b) in self.edges:
            if a not in self.nodes:
                self.nodes[a] = set()
            self.nodes[a].add(b)
            if b not in self.nodes:
                self.nodes[b] = set()
            self.nodes[b].add(a)


paths = []

def traverse(this_node, start_node, seen=None, level=0, graph=None, stop_level=3):
    level += 1
    local_seen = seen.copy()
    local_seen.append(this_node)
    if level > stop_level:
        return
    # Go to node and pick a new option
    for next_node in graph.nodes[this_node]:
        if next_node in local_seen:
            continue

        #print(start_node, local_seen, next_node)
        paths.append( local_seen.copy() + [next_node] )
        
        traverse(next_node, start_node, seen=local_seen, level=level, graph=graph)
        
    return False

skip_edges = [('jrc', 'qgc')] #, ('ppv', 'qgc')]
g = Graph(skip_edges=skip_edges)
for line in lines:
    g.process_str(line)

g.finalize()

print(datetime.datetime.now())
for i,n in enumerate(g.nodes):
    print(f"Working start node: {n} {i+1}/{len(g.nodes)}...")
    seen = [] #[n]
    traverse(n, n, seen=seen, level=0, graph=g)

# The results will have the longest paths... I'm guessing this path will have to traverse at least one bridge
#q = pd.Series(results)
#big_start, big_end = q[q==q.max()].head(1).index.tolist()[0]
print("Finished initial processing, finding trim_edges on num paths:", len(paths))
ctr = Counter()
for nodes in paths:
    for pair in list(zip(nodes, nodes[1:])):
        ctr[tuple(sorted(list(pair)))] += 1
print(pd.Series(ctr).sort_values(ascending=False).head(10))

trim_edges = pd.Series(ctr).sort_values(ascending=False).head((3-len(skip_edges))).index.tolist()

2023-12-27 00:50:01.277333
Working start node: pbc 1/1484...
Working start node: bnx 2/1484...
Working start node: trg 3/1484...
Working start node: chr 4/1484...
Working start node: mxt 5/1484...
Working start node: tkd 6/1484...
Working start node: dfd 7/1484...
Working start node: znt 8/1484...
Working start node: drl 9/1484...
Working start node: gkk 10/1484...
Working start node: kjs 11/1484...
Working start node: cbq 12/1484...
Working start node: flj 13/1484...
Working start node: qkp 14/1484...
Working start node: vpd 15/1484...
Working start node: nbr 16/1484...
Working start node: sms 17/1484...
Working start node: vrx 18/1484...
Working start node: jrc 19/1484...
Working start node: qgr 20/1484...
Working start node: qtx 21/1484...
Working start node: dqm 22/1484...
Working start node: hds 23/1484...
Working start node: dzt 24/1484...
Working start node: qgp 25/1484...
Working start node: qfm 26/1484...
Working start node: nql 27/1484...
Working start node: fxd 28/1484...
Wo

In [23]:
if True:
    skip_edges = [('jrc', 'qgc'), ('ppv', 'qgc')] #('jjs', 'vhf')]
    trim_edges = [('rkl', 'stt')]
print(datetime.datetime.now())
print("TRIMMING:", trim_edges)
new_g = Graph(skip_edges=skip_edges)
for edge in g.edges:
    if (edge not in trim_edges) and ( (edge[1], edge[0]) not in trim_edges):
        new_g.add_edge(edge)
new_g.finalize()

print("Final traverse")
# Finally, traverse from one side of one of the trimmed edges and count nodes:
seen = [] 
paths = []
#traverse(trim_edges[0][0], trim_edges[0][0], seen=seen, level=0, graph=new_g, stop_level=4)

for i,n in enumerate(new_g.nodes):
    print(f"Working start node: {n} {i+1}/{len(new_g.nodes)}...")
    traverse(n, n, seen=seen, level=0, graph=new_g, stop_level=4)

total = set()
for v in paths:
    total.update(v)
print(len(total), len(g.nodes))

print(datetime.datetime.now())

2023-12-27 00:53:12.059291
TRIMMING: [('rkl', 'stt')]
Final traverse
Working start node: pbc 1/1484...
Working start node: bnx 2/1484...
Working start node: trg 3/1484...
Working start node: chr 4/1484...
Working start node: mxt 5/1484...
Working start node: tkd 6/1484...
Working start node: dfd 7/1484...
Working start node: znt 8/1484...
Working start node: drl 9/1484...
Working start node: gkk 10/1484...
Working start node: kjs 11/1484...
Working start node: cbq 12/1484...
Working start node: flj 13/1484...
Working start node: qkp 14/1484...
Working start node: vpd 15/1484...
Working start node: nbr 16/1484...
Working start node: sms 17/1484...
Working start node: vrx 18/1484...
Working start node: jrc 19/1484...
Working start node: qgr 20/1484...
Working start node: qtx 21/1484...
Working start node: dqm 22/1484...
Working start node: hds 23/1484...
Working start node: dzt 24/1484...
Working start node: gtm 25/1484...
Working start node: pht 26/1484...
Working start node: rts 27/148

In [24]:
main_group = set(paths[0])
other_group = set()
for p in paths[1:]:
    if set(p).intersection(main_group):
        main_group.update(p)
    else:
        other_group.update(p)    

In [26]:
len(g.edges)

3310

In [25]:
len(main_group)

1484

In [ ]:
TRIMMING: [('ttx', 'xgv'), ('dgp', 'xgv'), ('vgt', 'xgv')]
1300 1484
2023-12-26 22:41:59.362733
TRIMMING: [('jrc', 'qgc'), ('ppv', 'qgc'), ('dgx', 'jrc')]   (level < 6)
773 1484
2023-12-26 22:41:59.675094
TRIMMING: [('jrc', 'qgc'), ('ppv', 'qgc'), ('jrc', 'krm')] (level < 8)
804 1484
2023-12-26 22:50:42.493680
Finished initial processing, finding trim_edges on num paths: 235631792
2023-12-27 00:31:20.206237  [('jrc', 'qgc'), ('ppv', 'qgc') (level < 9)
TRIMMING: [('rkl', 'stt')]
1227 1484 (this is junk)
2023-12-27 00:38:48.039226
TRIMMING: [('jjs', 'vhf'), ('rkl', 'stt')]
Final traverse
727 1484


In [8]:
pd.Series(ctr).sort_values(ascending=False).head(5)

rkl  stt    1364494
jjs  vhf    1363810
dml  vgk    1341802
qjb  vgk    1334790
gtg  qjb    1333106
dtype: int64

In [ ]:
x=0
for p in paths:
    x = max(len(p), x)
print(x)

In [ ]:
g.nodes['lhk']

In [ ]:
q = pd.Series(results)
big_start, big_end = q[q==q.max()].head(1).index.tolist()[0]

In [ ]:
eee = pd.Series(eccentricity)

In [ ]:
eee.sort_values().head(15)

In [ ]:
q.xs('hfx',level=1).sort_values()

In [ ]:
es = pd.Series(ecc_sum)
es.sort_values().head(5)

In [ ]:
sorted(ecc_sum.items())

In [ ]:
pgraph = pydot.Dot(graph_type='graph')

In [ ]:
for n1,n2 in g.edges:
    print(n1,n2)
    pgraph.add_edge(pydot.Edge(n1,n2))

In [ ]:
pgraph.write_png('graph25.png')

In [ ]:
# For each edge, find the SHORTEST cycle available


In [ ]:
def traverse(this_node, start_node, seen=None, level=0):
    level += 1
    local_seen = seen.copy()
    #print(start_node, this_node, "seen:", seen)
    # Go to node and pick a new option
    next_options = [next_node for next_node in g.nodes[this_node] if next_node not in local_seen]
#     if start_node=='bvb':
#         print(f"{this_node}... NEXT OPTIONS", next_options)
    for next_node in next_options.copy():
        #print(start_node, ["N"]*(level+1), next_node)
        print(start_node, local_seen, next_node)
#         if start_node=='bvb' and next_node=='frs':
#             print(start_node, next_node, results.get((start_node, next_node),999999), level)
        local_seen.append(next_node)
        if level < results.get((start_node, next_node),999999):
            results[(start_node, next_node)] = min(results.get((start_node, next_node),999999), level)
            paths[(start_node, next_node)] = local_seen.copy()
        
        traverse(next_node, start_node, seen=local_seen, level=level)
    return
